# KWIAT Entanglement Pair


What this does: Introduces the end-to-end simulation: pump prep → two-crystal type-I SPDC → per-arm analyzers → PBS → four coincidence channels.This notebook contains detailed Markdown before every block, mapping process ↔ hardware ↔ code.

| Process                      | Lab hardware                           | Code functions                            | Key params                              |
| ---------------------------- | -------------------------------------- | ----------------------------------------- | --------------------------------------- |
| Pump polarization            | 402 nm diode/laser, polarizer, HWP/QWP | `H`, `V`, `rho_pump`                      | purity $p$                              |
| Pump pre-optics/compensation | Quartz plates, tiltable compensators   | `jones_quartz`, `stack` → `U_pump`        | $L,\ \Delta n,\ \theta$                 |
| Type-I SPDC (two crystals)   | Two orthogonal type-I BBO              | `pair_state_from_two_crystals`            | $\theta_1,\theta_2,\kappa,\phi$         |
| Filtering/mode match         | IFs, irises, SMF coupling              | `add_white_noise`                         | noise $w$, overlap $\kappa$             |
| Per-arm analyzers            | QWP+HWP on rotation mounts             | `analyzer_stack`, `apply_local_analyzers` | $\alpha,\ \beta$                        |
| PBS split                    | PBS cubes                              | `pbs_projectors`, `joint_probs`           | PBS angle (fixed 0°)                    |
| Detection & counting         | APDs/SPADs, time-tagger                | `coincidence_counts`                      | $R,T,\eta_A,\eta_B$, darks, accidentals |
| CHSH evaluation              | Settings cycling                       | `compute_chsh_grid`                       | $\alpha,\alpha',\beta,\beta'$           |


## 1) Linear algebra/ Lab setup
---

**What this does:** Defines basis vectors, rotations, normalization, and Kronecker products.

**Mathematical process:**

Basis and identities:
$$
|H\rangle=(1,0)^{\mathsf T},\quad |V\rangle=(0,1)^{\mathsf T},\quad I_2,\ I_4.
$$

Normalization:
$$
\mathrm{ket}(v)=\frac{v}{\|v\|}.
$$

Rotation between lab and crystal frames:
$$
R(\theta)=
\begin{pmatrix}
\cos\theta & \sin\theta\\
-\sin\theta & \cos\theta
\end{pmatrix}.
$$

Kronecker product to build two-photon operators/states:
$$
A\otimes B.
$$

**Lab equipment:** conceptual choice of axes.
---

In [9]:
import numpy as np

# Lab basis and identities (be explicit about dtype)

H = np.array([1.0, 0.0], dtype=complex)     # |H>
V = np.array([0.0, 1.0], dtype=complex)     # |V>
I2 = np.eye(2, dtype=complex)               # 2×2 identity (complex)
I4 = np.eye(4, dtype=complex)               # 4×4 identity (complex)
#defining the normalized states
def ket(v: np.ndarray) -> np.ndarray:
    '''returns the normalized state
     inputs: v -> ndarray
    outputs :v/|v| -> ndarray'''
    v = np.asarray(v, dtype=complex).reshape(-1,1)
    n = np.linalg.norm(v)
    return v/n if n > 0 else v
# defining the rotational matrices
def R(theta_rad: float) -> np.ndarray:
    ''' 2d rotational matrix. Rotates the vector about the z axis along theta
    inputs : theta -> rad scalar
    outputs : R -> matrix nd.array'''
    c, s = np.cos(theta_rad), np.sin(theta_rad)
    return np.array([[c, s], [-s, c]], dtype=complex)
# defining the kronecker product.
def kron(*ops: np.ndarray) -> np.ndarray:
    ''' Kronecker product, to go from operators in the single particle
    system to a multiple particle system.
    input: ops nd.array 2x2 matrix
    output: ops \\xor ops -> nd.array 4x4 or 2^n x 2^n matrix depending on n particles'''
    out = np.array([[1.0+0j]], dtype=complex)
    for op in ops:
        out = np.kron(out, op)
    return out


## 2) Jones optics and crystal axes
Jones calculus models, important for the experiment are defined here.
---

**What this does:** Implements crystal eigen-axes and ideal retarders (QWP/HWP/quartz).

**Mathematical process:**

Crystal axes for a crystal rotated by $\theta$ relative to lab $H$:
$$
|o_\theta\rangle=\cos\theta\,|H\rangle+\sin\theta\,|V\rangle,\qquad
|e_\theta\rangle=-\sin\theta\,|H\rangle+\cos\theta\,|V\rangle.
$$

Linear retarder with fast axis $\theta$ and retardance $\delta$:
$$
J(\delta,\theta)=R(-\theta)\,\mathrm{diag}\!\bigl(1,e^{i\delta}\bigr)\,R(\theta).
$$

Special cases: QWP $\delta=\pi/2$, HWP $\delta=\pi$.

Quartz plate phase for wavelength $\lambda$ (nm), thickness $L$ (mm), birefringence $\Delta n$:
$$
\delta=\frac{2\pi\,\Delta n\,L}{\lambda}\quad\text{(with }L\text{ converted to nm)}.
$$

Cascaded optics multiply in propagation order: $U_{\text{stack}} = M_k \cdots M_2 M_1$.


**Lab equipment:** QWP, HWP, quartz plates on rotation/tilt mounts.
---

In [10]:
def basis_oe(th_deg):
    ''' returns the kets based on the crystal's basis.
    inputs: theta -> orientation of the crystal along the lab axis
    outputs: o,e -> kets'''
    import numpy as _np
    th=_np.deg2rad(th_deg); o=_np.cos(th)*H+_np.sin(th)*V; e=-_np.sin(th)*H+_np.cos(th)*V
    return ket(o), ket(e)
def jones_retarder(delta,th_deg):
    ''' Template for a QWP/HWP, Retarding plates.
    Rotates the state polarization and adds a delay in the slow axis.
    inputs:
    delta -> Phase delay
    theta -> rotational angle
    outputs: U -> ndarray'''
    import numpy as _np
    th=_np.deg2rad(th_deg); from numpy import exp
    return R(-th) @ _np.diag([1+0j, exp(1j*delta)]) @ R(th)
def jones_qwp(th_deg):  return jones_retarder(np.pi/2, th_deg)#implementation of QWP
def jones_hwp(th_deg): return jones_retarder(np.pi, th_deg)#implementation of HWP
def jones_quartz(lam_nm, delta_n, L_mm, th_deg):
  ''' Implementation of the Quartz crystal
  inputs :
          lam_nm -> Wavelength
          delta_n -> Birefringence.
          L_mm    -> Thickness.
          th_deg -> Rotational angle.
  outputs: U -> ndarray'''
  import numpy as _np
  delta=2*_np.pi*delta_n*(L_mm*1e6)/lam_nm;
  return jones_retarder(delta, th_deg)
def stack(*mats):
    import numpy as _np
    U=_np.eye(2,dtype = complex)
    for M in mats: U=M @ U
    return U

## 3) SPDC source
---

**What this does:** Builds the two-photon polarization density matrix from a mixed pump after compensation for two orthogonal type-I crystals.

**Mathematical process:**

Pump pre-optics:
$$
\rho_p' = U_p\,\rho_p\,U_p^\dagger.
$$

Overlaps on extraordinary axes set amplitudes:
$$
a=\langle e_{\theta_1}|\rho_p'|e_{\theta_1}\rangle,\qquad
b=\langle e_{\theta_2}|\rho_p'|e_{\theta_2}\rangle,\qquad
c=\langle e_{\theta_1}|\rho_p'|e_{\theta_2}\rangle.
$$

Two-photon state in the span $\{|o_{\theta_1}o_{\theta_1}\rangle,\ |o_{\theta_2}o_{\theta_2}\rangle\}$:
$$
\rho \propto
a\,|o_1o_1\rangle\!\langle o_1o_1|
+ b\,|o_2o_2\rangle\!\langle o_2o_2|
+ \kappa\,c\,e^{-i\phi}\,|o_1o_1\rangle\!\langle o_2o_2|
+ \kappa\,c^{*}e^{i\phi}\,|o_2o_2\rangle\!\langle o_1o_1|.
$$

Normalize: $\mathrm{Tr}\,\rho=1$.

Depolarizing noise (mode mismatch, spectral impurity):
$$
\rho \leftarrow (1-w)\,\rho + w\,\frac{I_4}{4}.
$$

**Lab equipment:** two orthogonal BBO crystals; quartz compensators set $\phi$; filters and fiber coupling control $\kappa$ and motivate $w$.

---

In [11]:
def pump_after_optics(rho,U):
   ''' Readies the pump after to go into the SPDC, adds the delays, polarization, e.t.c.'''
   return U @ rho @ U.conj().T
def pair_state_from_two_crystals(rp,t1,t2,phi=0.0,kappa=1.0):
    ''' Downconversion state based on the Prepared Pump state.
    inputs:
            rp = rho prime -> nd array prepared pump ready to be downconverted
            t1: crystal axis 1
            t2: crystal axis 2
            phi: Phase difference between the 2 pair production schemes.
            kappa: The coherence term between the 2 states.
    Process: Calculates the coefficients of the states by taking inner products.
    Outputs:
            rho: Downconverted state.

    '''
    o1,e1=basis_oe(t1); o2,e2=basis_oe(t2)
    a=(e1.conj().T @ rp @ e1).item(); b=(e2.conj().T @ rp @ e2).item(); c=(e1.conj().T @ rp @ e2).item()
    psi1=kron(o1,o1); psi2=kron(o2,o2)
    rho= a*(psi1@psi1.conj().T) + b*(psi2@psi2.conj().T) + kappa*c*np.exp(-1j*phi)*(psi1@psi2.conj().T) + kappa*np.conj(c)*np.exp(1j*phi)*(psi2@psi1.conj().T)
    tr=float(np.real(np.trace(rho))); return rho/tr if tr>0 else rho
def add_white_noise(r,w): return (1-w)*r + w*np.eye(4,dtype =complex)/4.0

## 4) Analyzers → PBS → probabilities/counts

**What this does:** Applies per-arm analyzer unitaries, then PBS projectors, then computes joint detection probabilities and converts them to coincidences.

**Mathematical process:**

Analyzer stacks per arm rotate the basis; state update:
$$
\rho'=(U_A\otimes U_B)\,\rho\,(U_A\otimes U_B)^\dagger.
$$

Linear projector onto $|\alpha\rangle=\cos\alpha\,|H\rangle+\sin\alpha\,|V\rangle$:
$$
P_\alpha = |\alpha\rangle\langle\alpha|.
$$

PBS at $0^\circ$ after analyzers: transmit $P$, reflect $I-P$.

Four joint outcomes:
$$
\begin{aligned}
P_{AB}   &= \mathrm{Tr}\!\big[\rho'\,(P_A\!\otimes\! P_B)\big],\\
P_{AB'}  &= \mathrm{Tr}\!\big[\rho'\,(P_A\!\otimes\! (I-P_B))\big],\\
P_{A'B}  &= \mathrm{Tr}\!\big[\rho'\,((I-P_A)\!\otimes\! P_B)\big],\\
P_{A'B'} &= \mathrm{Tr}\!\big[\rho'\,((I-P_A)\!\otimes\! (I-P_B))\big].
\end{aligned}
$$

Coincidence model with pair rate $R$, dwell $T$, efficiencies $\eta_A,\eta_B$:
$$
C_{xy}^{\text{true}} = R\,T\,\eta_A\,\eta_B\,P_{xy}.
$$

Accidental pedestal (simple proportional model):
$$
C_{xy} \leftarrow C_{xy} + \frac{\text{accidentals}}{4}\sum_{u,v} C_{uv}.
$$

Dark–dark floor:
$$
C_{xy} \leftarrow C_{xy} + \frac{1}{4}\,\text{dark}_A\,\text{dark}_B\,T.
$$

**Lab equipment:** QWP/HWP stacks feeding PBS cubes; four detector channels into a time-tagger.

---


In [12]:
def analyzer_stack(lambda_nm: float, alpha: float,
                   qwp_deg: float = 0.0, add_quartz: bool = False,
                   delta_n: float = 0.0093, L_mm: float = 0.0, theta_q: float = 0.0) -> np.ndarray:
    # HWP at alpha/2 rotates the linear measurement basis to alpha
    return jones_hwp(alpha / 2.0)

def apply_local_analyzers(rho,UA,UB): U=np.kron(UA,UB); return U @ rho @ U.conj().T
def projector_linear(alpha: float) -> np.ndarray:
    a = np.deg2rad(alpha)
    psi = ket(np.cos(a)*H + np.sin(a)*V)
    return psi @ psi.conj().T

def pbs_projectors():  # no angle; analyzer already set basis
    P_H = projector_linear(0.0)
    return P_H, (I2 - P_H)

def joint_probs(rho: np.ndarray, Pa: np.ndarray, Pb: np.ndarray):
    Pa_perp = I2 - Pa
    Pb_perp = I2 - Pb
    ops = {
        ("A","B"):   np.kron(Pa,      Pb),
        ("A","B'"):  np.kron(Pa,      Pb_perp),
        ("A'","B"):  np.kron(Pa_perp, Pb),
        ("A'","B'"): np.kron(Pa_perp, Pb_perp),
    }
    return {k: float(np.real(np.trace(rho @ O))) for k, O in ops.items()}

def coincidence_counts(P,R,T,etaA,etaB,darkA=0.0,darkB=0.0,accidentals=0.0):
    true=R*T*etaA*etaB; C={k:true*v for k,v in P.items()}
    if accidentals>0:
        tot=sum(C.values())
        C={k:v+accidentals*tot/4.0 for k,v in C.items()}
    dd=darkA*darkB*T; C={k:v+dd/4.0 for k,v in C.items()}; return C

## 5) High-level wrappers

**Pipeline math:**
$$
\rho_p \xrightarrow{U_p} \rho_p' \xrightarrow{\text{SPDC}} \rho
\xrightarrow{U_A\otimes U_B} \rho'
\xrightarrow{\text{PBS projectors}} \{P_{xy}\}
\xrightarrow{\text{count model}} \{C_{xy}\}.
$$

**Outputs:** dictionaries for probabilities $P_{xy}$ and counts $C_{xy}$ with $x\in\{A,A'\}$, $y\in\{B,B'\}$.  
**Lab equipment:** fix mounts at $\alpha,\beta$ and record the four coincidence channels.

---

In [13]:
def compute_four_ports_for_settings(rho_pump,U_pump,t1,t2,kappa,phi,white_noise,alpha,beta,lambda_s=804.0,lambda_i=804.0,
                                R=1e5,T=1.0,etaA=0.6,etaB=0.6,darkA=0.0,darkB=0.0,accidentals=0.0):
    # prepare the pump and downconverted state
    rp = pump_after_optics(rho_pump, U_pump) if U_pump is not None else rho_pump
    # enforce perfect overlap (kappa) and no white noise by default
    rho_pairs = pair_state_from_two_crystals(rp, t1, t2, phi=phi, kappa=kappa)
    rho_pairs = add_white_noise(rho_pairs, white_noise)
    # apply local analyzers (HWP only for linear bases)
    UA = analyzer_stack(lambda_s, alpha)
    UB = analyzer_stack(lambda_i, beta)
    rho_after = apply_local_analyzers(rho_pairs, UA, UB)
    # project onto fixed lab H/V at the PBS
    Pa = projector_linear(0.0)
    Pb = projector_linear(0.0)
    P = joint_probs(rho_after, Pa, Pb)
    C = coincidence_counts(P, R, T, etaA, etaB, darkA, darkB, accidentals)
    return P, C

## 6) Example run and output table

**What this does:** Chooses practical defaults and prints the four channels.

**Mathematical process:**

Pump state:
$$
\rho_p = p\,|45^\circ\rangle\langle45^\circ| + (1-p)\,\frac{I_2}{2}.
$$

Crystal axes and source parameters:
$$
\theta_1=0^\circ,\quad \theta_2=90^\circ,\quad \kappa\in[0,1],\quad \phi\in\mathbb{R},\quad w\in[0,1].
$$

Analyzer settings: $\alpha,\ \beta$ (degrees).

Outputs computed:
$$
\{P_{AB}, P_{AB'}, P_{A'B}, P_{A'B'}\},\qquad
\{C_{AB}, C_{AB'}, C_{A'B}, C_{A'B'}\}.
$$

**Lab equipment:** set angles on rotation mounts; count coincidences for $T$ seconds.

---

In [14]:
import numpy as np
import pandas as pd

# Lab basis and identities (be explicit about dtype)
H = np.array([1.0, 0.0], dtype=complex)     # |H>
V = np.array([0.0, 1.0], dtype=complex)     # |V>
I2 = np.eye(2, dtype=complex)               # 2×2 identity (complex)



# Generate alpha and beta arrays from 0 to 2*pi with increments of pi/8
alpha_values = np.arange(0, 2 * np.pi, np.pi / 8)
beta_values = np.arange(0, 2 * np.pi, np.pi / 8)

order=[('A','B'),('A','B\''),('A\'','B'),('A\'','B\'')]
data = []

for alpha in alpha_values:
    for beta in beta_values:
        psi45=(H+V)/np.sqrt(2); rho_pure=(psi45.reshape(-1,1) @ psi45.reshape(1,-1).conj());p = 0.85; rho_pump=p*rho_pure+(1-p)*I2/2
        # use perfect overlap (kappa=1) and zero white noise to allow entanglement
        theta1, theta2 = 0.0, -90.0
        kappa, phi, white_noise = 0.9, 0.0, 0.02
        U_pump=I2
        # set dark counts and accidentals to zero to maximise correlation for CHSH test
        P,C=compute_four_ports_for_settings(
            rho_pump,U_pump,theta1,theta2,kappa,phi,white_noise,
            np.rad2deg(alpha),np.rad2deg(beta),
            R=1e5,T=1.0,etaA=1.0,etaB=1.0,
            darkA=0.0,darkB=0.0,accidentals=0.0)
        for k in order:
            data.append({
                'alpha': np.rad2deg(alpha),
                'beta': np.rad2deg(beta),
                'Detector pair': f"{k[0]}-{k[1]}",
                'Probability': P[k],
                'Expected coincidences': C[k]
            })

df = pd.DataFrame(data)
df

,alpha,beta,Detector pair,Probability,Expected coincidences
0,0.0,0.0,A-B,0.495000,49500.000000
1,0.0,0.0,A-B',0.005000,500.000000
2,0.0,0.0,A'-B,0.005000,500.000000
3,0.0,0.0,A'-B',0.495000,49500.000000
4,0.0,22.5,A-B,0.423241,42324.116139
...,...,...,...,...,...
1019,337.5,315.0,A'-B',0.382529,38252.948846
1020,337.5,337.5,A-B,0.466212,46621.250000
1021,337.5,337.5,A-B',0.033787,3378.750000
1022,337.5,337.5,A'-B,0.033787,3378.750000


In [15]:
import numpy as np
import pandas as pd

Detector_pair = ["A-B", "A-B'", "A'-B", "A'-B'"]

def _block_counts(df, a, b, value_col="Expected coincidences"):
    """Return a dict of port -> counts for a single (alpha,beta) block."""
    block = df[(df['alpha'] == a) & (df['beta'] == b)]
    if block.empty:
        raise ValueError(f"No data for alpha={a}, beta={b}")
    counts = dict(zip(block['Detector pair'], block[value_col]))
    missing = [p for p in Detector_pair if p not in counts]
    if missing:
        raise ValueError(f"Missing ports at (alpha={a}, beta={b}): {missing}")
    return counts

def _E_and_sigma_from_counts(counts):
    """
    Compute correlation E and its standard error from raw counts.
    Uses the ±1 estimator: E = (N_corr - N_anti)/N with
    var(E) ≈ (1 - E^2)/N for independent trials.
    """
    N_AB   = float(counts["A-B"])
    N_ABp  = float(counts["A-B'"])
    N_ApB  = float(counts["A'-B"])
    N_ApBp = float(counts["A'-B'"])
    N_corr = N_AB + N_ApBp
    N_anti = N_ABp + N_ApB
    N_tot  = N_corr + N_anti
    if N_tot <= 0:
        return 0.0, np.inf
    E = (N_corr - N_anti) / N_tot
    sigma_E = np.sqrt(max(0.0, 1.0 - E**2) / N_tot)
    return E, sigma_E

def CHSH_S(df, alpha, alpha_p, beta, beta_p, value_col="Expected coincidences"):
    """Compute S and its propagated uncertainty for chosen settings."""
    Eab,  sab  = _E_and_sigma_from_counts(_block_counts(df, alpha,   beta,   value_col))
    Eabp, sabp = _E_and_sigma_from_counts(_block_counts(df, alpha,   beta_p, value_col))
    Eapb, sapb = _E_and_sigma_from_counts(_block_counts(df, alpha_p, beta,   value_col))
    Eapbp,sapbp= _E_and_sigma_from_counts(_block_counts(df, alpha_p, beta_p, value_col))
    S = Eab - Eabp + Eapb + Eapbp
    sigma_S = np.sqrt(sab**2 + sabp**2 + sapb**2 + sapbp**2)
    return S, sigma_S

# ---------- Option A: Use the textbook “max-violation” angles ----------
alpha, alpha_p = 0.0, 45.0
beta,  beta_p  = 22.5, 67.5

S, sS = CHSH_S(df, alpha, alpha_p, beta, beta_p, value_col="Expected coincidences")
zscore = (abs(S) - 2.0)/sS if np.isfinite(sS) else np.nan
print(f"S = {S:.4f} ± {sS:.4f}   →  {'VIOLATION' if abs(S) > 2 else 'no violation'}; "
      f"excess over 2 by {zscore:.2f}σ")

# ---------- Option B: Scan your entire grid to find the strongest violation ----------
alphas = sorted(df['alpha'].unique())
betas  = sorted(df['beta'].unique())

records = []
for i in range(len(alphas)):
    for ip in range(i+1, len(alphas)):   # distinct alpha, alpha'
        for j in range(len(betas)):
            for jp in range(j+1, len(betas)):  # distinct beta, beta'
                a, ap = alphas[i], alphas[ip]
                b, bp = betas[j],  betas[jp]
                try:
                    S, sS = CHSH_S(df, a, ap, b, bp, value_col="Expected coincidences")
                    records.append({
                        "alpha": a, "alpha_p": ap, "beta": b, "beta_p": bp,
                        "S": S, "sigma_S": sS, "absS": abs(S),
                        "violated": abs(S) > 2,
                        "z_over_2": (abs(S)-2)/sS if np.isfinite(sS) else np.nan
                    })
                except ValueError:
                    # skip incomplete blocks
                    continue

scan = pd.DataFrame(records).sort_values("absS", ascending=False)
print(scan.head(100)[["alpha","alpha_p","beta","beta_p","S","sigma_S","violated","z_over_2"]])

scan.to_csv('scan.csv')

S = 2.4462 ± 0.0050   →  VIOLATION; excess over 2 by 89.64σ
       alpha  alpha_p   beta  beta_p         S   sigma_S  violated   z_over_2
9163   135.0    180.0   67.5   112.5 -2.446165  0.004977      True  89.636417
6213    67.5    292.5  180.0   225.0 -2.446165  0.004977      True  89.636417
5215    67.5    112.5   90.0   135.0  2.446165  0.004977      True  89.636417
5253    67.5    112.5  180.0   225.0 -2.446165  0.004977      True  89.636417
5223    67.5    112.5   90.0   315.0  2.446165  0.004977      True  89.636417
...      ...      ...    ...     ...       ...       ...       ...        ...
8413   112.5    247.5    0.0   315.0 -2.446165  0.004977      True  89.636417
10760  157.5    292.5  135.0   270.0  2.446165  0.004977      True  89.636417
136      0.0     45.0   22.5    67.5  2.446165  0.004977      True  89.636417
1649     0.0    315.0  157.5   292.5  2.446165  0.004977      True  89.636417
8405   112.5    247.5    0.0   135.0 -2.446165  0.004977      True  89.636417

[10

In [16]:
import numpy as np
H = np.array([1,0], complex); V = np.array([0,1], complex)
I2 = np.eye(2, dtype=complex)

def ket(v): v=np.asarray(v,complex).reshape(-1,1); return v/np.linalg.norm(v)

# set pump purity p
p = 1.0  # start with 1.0; later replace with your measured DOP (e.g., 0.98)
psi45 = ket(H + V)
rho_pure = psi45 @ psi45.conj().T
rho_pump = p*rho_pure + (1-p)*I2/2

# quick threshold check given your kappa and white_noise (w)
kappa = 0.95
w = 0.02
p_min = (1/np.sqrt(2)) / ((1 - w) * kappa)
print("p_min for CHSH violation ≈", p_min)


p_min for CHSH violation ≈ 0.7595131913926396
